**Transfer Learning**, often abbreviated as TL, is a strategy used in deep learning.

In this approach, the knowledge or understanding gained from one task is applied to a similar but different task to improve its performance.

To give you an example, let's consider image classification.

If a machine learning model has learned to identify cars, that knowledge can be transferred and used when the model is trying to identify trucks.

This is because both tasks are related and share common features.

So, the model doesn't have to start learning from scratch for the new task, which makes the learning process more efficient.

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert X_train.shape == (50000, 32, 32, 3)
assert X_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
# Convert the data to TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 255.0)
    image = tf.image.resize(image, (224,224))
    return image, label

# Dataset preparation function with augmentation option
def prepare_dataset(ds, batch_size=20):
    ds = ds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=10000).batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

# Preprocess the data
train_ds = prepare_dataset(train_ds)
test_ds = prepare_dataset(test_ds)

In [ ]:
import tensorflow_hub as hub

model_url = "https://www.kaggle.com/models/google/efficientnet-v2/TensorFlow2/imagenet1k-b0-feature-vector/2"

model = tf.keras.Sequential(
    [
        hub.kerasLayer(model_url, trainable=False, input_shape=(224, 224, 3), name="feature_extraction_layer"),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dropout(rate=0.2),
        tf.keras.layers.Dense(10, activation="softmax")
    ]
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
model.summary()

In [ ]:
# Train the model
history = model.fit(
    train_ds, epochs=100, validation_data=test_ds, batch_size=20)

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()